In [12]:
#Fichero nuevo para conectarnos al MySQL 


from dotenv import load_dotenv
import pandas as pd
import os
from sqlalchemy import create_engine

load_dotenv()  # Carga las variables de entorno del archivo .env

# Datos de conexión
usuario = os.getenv('DB_USER')
contraseña = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
puerto = os.getenv('DB_PORT')
base_datos = os.getenv('DB_NAME')

# Crear el motor de conexión con SQLAlchemy y mysql-connector
conexion_str = f"mysql+mysqlconnector://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}"
conexion = create_engine(conexion_str)
print("Conectado successfully")  

#Voy a cargar en un DataFrame los datos de la tabla Pedidos
#Ver todos los campos de la tabla
pedidos = pd.read_sql("Select * from Pedidos",conexion)
#Ver un solo campo de la tabla
pedidos = pd.read_sql("Select FechaPedido from Pedidos",conexion)
#Ver mas de un campo de la tabla
pedidos = pd.read_sql("Select FechaPedido, Destinatario from Pedidos",conexion)
#Poner un alias para el campo
pedidos = pd.read_sql("Select FechaPedido as Fecha, " \
"Destinatario as cliente from Pedidos",conexion)
#Como hacemos la ordenación, por defecto es ASC es decir Ascendente
pedidos = pd.read_sql("Select * from Pedidos order by Destinatario",conexion)
#Como hacemos la ordenación, por defecto es ASC es decir Ascendente
pedidos = pd.read_sql("Select * from Pedidos order by Destinatario desc",conexion)
#El filtrado con WHERE
pedidos = pd.read_sql("Select * from Pedidos where PaísDestinatario ='Alemania'",conexion)
#Filtrar + ordenar en este orden, primero filtro y despues ordeno
pedidos = pd.read_sql("Select * from Pedidos where PaísDestinatario ='Alemania'" \
" order by Destinatario",conexion)
#La unica manera de filtrar en el sujeto con Top
pedidos = pd.read_sql("Select * from Pedidos LIMIT 3", conexion)
#Como hacemos si el nombre de la tabla tiene espacios en blanco, con corchete delante y detras del
#nombre
pedidos = pd.read_sql("Select * from `Detalles de pedidos`",conexion)
#Voy a calcular el importe en la sentencia SQL
pedidos = pd.read_sql("Select PrecioUnidad, Cantidad, (PrecioUnidad * Cantidad) as Importe from `Detalles de pedidos`",conexion)
pedidos


Conectado successfully


,PrecioUnidad,Cantidad,Importe
0,14.00,12,168.0
1,9.80,10,98.0
2,34.80,5,174.0
3,18.60,9,167.4
4,42.40,40,1696.0
...,...,...,...
2150,33.25,2,66.5
2151,17.00,1,17.0
2152,15.00,2,30.0
2153,7.75,4,31.0


In [ ]:
#Como ver que drivers soporta pyodbc en este PC
import pyodbc
pyodbc.drivers()


PRACTICA 33:
Cargaremos del servidor de SQL Server 2 dataframes que contengan por separado, los pedidos solamente del pais Francia y los detalles de pedidos. Realizaremos las transformaciones necesarias
para tener en un dataframe la suma de importe (PrecioUnidad x Cantidad) por CiudadDestinatario, lo podemos calcular en el dataframe despues de merge

In [ ]:
#Practica 33
from dotenv import load_dotenv
import pandas as pd
import os
from sqlalchemy import create_engine

load_dotenv()  # Carga las variables de entorno del archivo .env

# Datos de conexión
usuario = os.getenv('DB_USER')
contraseña = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
puerto = os.getenv('DB_PORT')
base_datos = os.getenv('DB_NAME')

# Crear el motor de conexión con SQLAlchemy y mysql-connector
conexion_str = f"mysql+mysqlconnector://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}"
conexion = create_engine(conexion_str)
print("Conectado successfully")  

#Cargar los dataframes
SQL = "select IdPedido, CiudadDestinatario from Pedidos where PaísDestinatario='Francia'"
pedidos_francia = pd.read_sql(SQL,conexion)
SQL = "select IdPedido, PrecioUnidad, Cantidad from `Detalles de pedidos`"
detalles_pedidos = pd.read_sql(SQL,conexion)
#hacer el merge
resultado = pd.merge(pedidos_francia, detalles_pedidos, on="IdPedido", how="inner")
#Crear la columna o campo nuevo, importe
resultado['Importe'] = resultado['PrecioUnidad']*resultado['Cantidad']
#la agrupacion
resultado.groupby(['CiudadDestinatario'])['Importe'].sum()
resultado_agrup = resultado.groupby(['CiudadDestinatario'])['Importe'].sum()
resultado_agrup = pd.DataFrame(resultado_agrup, columns=['Importe'])
#Para generar el indice numerico a la vez que Category se convierte en una columna
resultado_agrup = resultado_agrup.reset_index()
resultado_agrup

Conectado successfully


,CiudadDestinatario,Importe
0,Lille,11666.90
1,Lyon,9937.10
2,Marseille,23850.95
3,Nantes,4788.06
4,Paris,2423.35
5,Reims,1040.00
6,Strasbourg,19088.00
7,Toulouse,10272.35
8,Versailles,1992.05


In [ ]:
from dotenv import load_dotenv
import pandas as pd
import os
from sqlalchemy import create_engine

load_dotenv()  # Carga las variables de entorno del archivo .env

# Datos de conexión
usuario = os.getenv('DB_USER')
contraseña = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
puerto = os.getenv('DB_PORT')
base_datos = os.getenv('DB_NAME')

# Crear el motor de conexión con SQLAlchemy y mysql-connector
conexion_str = f"mysql+mysqlconnector://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}"
conexion = create_engine(conexion_str)
print("Conectado successfully")  

#Cargar los dataframes, voy a coger de la sentencia SQL 2 tablas, necesito hacer un join
#vamos a hacer un join implicito = inner join
SQL = "Select * from Pedidos, `Detalles de pedidos` where Pedidos.IdPedido = " \
"[Detalles de pedidos].IdPedido"
#Vamos a mejorar la sentencia, pondremos alias de tablas
SQL = "Select * from Pedidos P, `Detalles de pedidos` DP where P.IdPedido = " \
"DP.IdPedido"
#Y ademas filtraremos por Francia
SQL = "Select * from Pedidos P, `Detalles de pedidos` DP where P.IdPedido = " \
"DP.IdPedido and P.PaísDestinatario = 'Francia'"
#Y ademas calculamos el importe
SQL = "Select P.CiudadDestinatario, (DP.PrecioUnidad * DP.Cantidad) as Importe from Pedidos P, " \
"`Detalles de pedidos` DP where P.IdPedido = " \
"DP.IdPedido and P.PaísDestinatario = 'Francia'"
#Y ademas me lo traigo ya agrupado por Ciudad
SQL = "Select P.CiudadDestinatario, SUM(DP.PrecioUnidad * DP.Cantidad) as Importe from Pedidos P, " \
"`Detalles de pedidos` DP where P.IdPedido = " \
"DP.IdPedido and P.PaísDestinatario = 'Francia' GROUP BY P.CiudadDestinatario"
pedidos = pd.read_sql(SQL,conexion)
pedidos

In [14]:
SQL = "Select * from Pedidos"
pedidos = pd.read_sql(SQL,conexion)
pedidos.iloc[0,0] = "10248"
pedidos


/var/folders/6f/lhr87vh51bq38sm9v46389m00000gn/T/ipykernel_65083/2858044096.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10248' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  pedidos.iloc[0,0] = "10248"


,IdPedido,IdCliente,IdEmpleado,FechaPedido,FechaEntrega,FechaEnvío,FormaEnvío,Cargo,Destinatario,DirecciónDestinatario,CiudadDestinatario,RegiónDestinatario,CódPostalDestinatario,PaísDestinatario
0,10248,WELLI,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Wilman Kala,Keskuskatu 45,Helsinki,None,21240,Finlandia
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Alemania
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Río de Janeiro,RJ,05454-876,Brasil
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,Francia
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Bélgica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,1998-05-05,1998-06-02,NaT,2,24.95,Pericles Comidas clásicas,Calle Dr. Jorge Cash 321,México D.F.,None,05033,México
826,11074,SIMOB,7,1998-05-06,1998-06-03,NaT,2,18.44,Simons bistro,Vinbæltet 34,København,None,1734,Dinamarca
827,11075,RICSU,8,1998-05-06,1998-06-03,NaT,2,6.19,Richter Supermarkt,Starenweg 5,Genève,None,1204,Suiza
828,11076,BONAP,4,1998-05-06,1998-06-03,NaT,2,38.28,Bon app',"12, rue des Bouchers",Marseille,None,13008,Francia


El objeto iLOC de un DataFrame, significa INTEGER LOCATION
me permite acceder al valor de una columna y fila concreta de un DataFrame, en sus 
modalidades lectura y escritura

In [15]:
#Los registros con los indices del 20 al 29, son 10 en total
pedidos.iloc[20:30,]

,IdPedido,IdCliente,IdEmpleado,FechaPedido,FechaEntrega,FechaEnvío,FormaEnvío,Cargo,Destinatario,DirecciónDestinatario,CiudadDestinatario,RegiónDestinatario,CódPostalDestinatario,PaísDestinatario
20,10268,GROSR,8,1996-07-30,1996-08-27,1996-08-02,3,66.29,GROSELLA-Restaurante,5ª Ave. Los Palos Grandes,Caracas,DF,1081,Venezuela
21,10269,WHITC,5,1996-07-31,1996-08-14,1996-08-09,1,4.56,White Clover Markets,1029 - 12th Ave. S.,Seattle,WA,98124,Estados Unidos
22,10270,WARTH,1,1996-08-01,1996-08-29,1996-08-02,1,136.54,Wartian Herkku,Torikatu 38,Oulu,None,90110,Finlandia
23,10271,SPLIR,6,1996-08-01,1996-08-29,1996-08-30,2,4.54,Split Rail Beer & Ale,P.O. Box 555,Lander,WY,82520,Estados Unidos
24,10272,RATTC,6,1996-08-02,1996-08-30,1996-08-06,2,98.03,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,Estados Unidos
25,10273,QUICK,3,1996-08-05,1996-09-02,1996-08-12,3,76.07,QUICK-Stop,Taucherstraße 10,Cunewalde,None,01307,Alemania
26,10274,VINET,6,1996-08-06,1996-09-03,1996-08-16,1,6.01,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,Francia
27,10275,MAGAA,1,1996-08-07,1996-09-04,1996-08-09,1,26.93,Magazzini Alimentari Riuniti,Via Ludovico il Moro 22,Bérgamo,None,24100,Italia
28,10276,TORTU,8,1996-08-08,1996-08-22,1996-08-14,3,13.84,Tortuga Restaurante,Avda. Azteca 123,México D.F.,None,05033,México
29,10277,MORGK,2,1996-08-09,1996-09-06,1996-08-13,3,125.77,Morgenstern Gesundkost,Heerstr. 22,Leipzig,None,04179,Alemania


In [16]:
pedidos.iloc[20:30,[4,2,3]]

,FechaEntrega,IdEmpleado,FechaPedido
20,1996-08-27,8,1996-07-30
21,1996-08-14,5,1996-07-31
22,1996-08-29,1,1996-08-01
23,1996-08-29,6,1996-08-01
24,1996-08-30,6,1996-08-02
25,1996-09-02,3,1996-08-05
26,1996-09-03,6,1996-08-06
27,1996-09-04,1,1996-08-07
28,1996-08-22,8,1996-08-08
29,1996-09-06,2,1996-08-09


In [17]:
#Copiar el indice en una columna nueva
pedidos['Copia_Indice'] = pedidos.index
pedidos

,IdPedido,IdCliente,IdEmpleado,FechaPedido,FechaEntrega,FechaEnvío,FormaEnvío,Cargo,Destinatario,DirecciónDestinatario,CiudadDestinatario,RegiónDestinatario,CódPostalDestinatario,PaísDestinatario,Copia_Indice
0,10248,WELLI,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Wilman Kala,Keskuskatu 45,Helsinki,None,21240,Finlandia,0
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Alemania,1
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Río de Janeiro,RJ,05454-876,Brasil,2
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,Francia,3
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Bélgica,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,1998-05-05,1998-06-02,NaT,2,24.95,Pericles Comidas clásicas,Calle Dr. Jorge Cash 321,México D.F.,None,05033,México,825
826,11074,SIMOB,7,1998-05-06,1998-06-03,NaT,2,18.44,Simons bistro,Vinbæltet 34,København,None,1734,Dinamarca,826
827,11075,RICSU,8,1998-05-06,1998-06-03,NaT,2,6.19,Richter Supermarkt,Starenweg 5,Genève,None,1204,Suiza,827
828,11076,BONAP,4,1998-05-06,1998-06-03,NaT,2,38.28,Bon app',"12, rue des Bouchers",Marseille,None,13008,Francia,828


In [18]:
#Crear un nuevo DataFrame con todas las filas y solamente algunas columnas
pedidos2 = pedidos.iloc[:,[6,3,1]]
pedidos2

,FormaEnvío,FechaPedido,IdCliente
0,3,1996-07-04,WELLI
1,1,1996-07-05,TOMSP
2,2,1996-07-08,HANAR
3,1,1996-07-08,VICTE
4,2,1996-07-09,SUPRD
...,...,...,...
825,2,1998-05-05,PERIC
826,2,1998-05-06,SIMOB
827,2,1998-05-06,RICSU
828,2,1998-05-06,BONAP


In [19]:
#Voy a ver el numero de los indices de la tabla
pedidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   IdPedido               830 non-null    object        
 1   IdCliente              830 non-null    object        
 2   IdEmpleado             830 non-null    int64         
 3   FechaPedido            830 non-null    datetime64[ns]
 4   FechaEntrega           830 non-null    datetime64[ns]
 5   FechaEnvío             809 non-null    datetime64[ns]
 6   FormaEnvío             830 non-null    int64         
 7   Cargo                  830 non-null    float64       
 8   Destinatario           830 non-null    object        
 9   DirecciónDestinatario  830 non-null    object        
 10  CiudadDestinatario     830 non-null    object        
 11  RegiónDestinatario     323 non-null    object        
 12  CódPostalDestinatario  811 non-null    object        
 13  PaísD

In [20]:
#Es que voy a poder recorrer todas las filas / columnas con un bucle
filas = len(pedidos)
filas
#como obtener el numero de columnas
columnas = len(pedidos.columns)
columnas
#Ya podria recorrer con un bucle
for i in range(0, filas):
    print("nombre",pedidos.iloc[i,1])

nombre WELLI
nombre TOMSP
nombre HANAR
nombre VICTE
nombre SUPRD
nombre HANAR
nombre CHOPS
nombre RICSU
nombre WELLI
nombre HILAA
nombre ERNSH
nombre CENTC
nombre OTTIK
nombre QUEDE
nombre RATTC
nombre ERNSH
nombre FOLKO
nombre BLONP
nombre WARTH
nombre FRANK
nombre GROSR
nombre WHITC
nombre WARTH
nombre SPLIR
nombre RATTC
nombre QUICK
nombre VINET
nombre MAGAA
nombre TORTU
nombre MORGK
nombre BERGS
nombre LEHMS
nombre BERGS
nombre ROMEY
nombre ROMEY
nombre LILAS
nombre LEHMS
nombre QUICK
nombre QUICK
nombre RICAR
nombre REGGC
nombre BSBEV
nombre COMMI
nombre QUEDE
nombre TRADH
nombre TORTU
nombre RATTC
nombre VINET
nombre LILAS
nombre BLONP
nombre HUNGO
nombre RICAR
nombre MAGAA
nombre WANDK
nombre SUPRD
nombre GODOS
nombre TORTU
nombre OLDWO
nombre ROMEY
nombre LONEP
nombre ANATR
nombre HUNGO
nombre THEBI
nombre DUMON
nombre WANDK
nombre QUICK
nombre RATTC
nombre ISLAT
nombre RATTC
nombre LONEP
nombre ISLAT
nombre TORTU
nombre WARTH
nombre ISLAT
nombre PERIC
nombre KOENE
nombre SAVEA